In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import os
import plotly.express as px


In [2]:
FILE_PATH = "/Users/evanlipton/Dropbox (YSE)/Solar_ElectricData/ISOs"

In [3]:
colors

UsageError: %colors: you must specify a color scheme. See '%colors?'


In [9]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f


In [10]:
ISO_dict = {}
for file in listdir_nohidden(FILE_PATH):
    temp = pd.read_csv(os.path.join(FILE_PATH, file), thousands = ",")
    temp["time_tomerge"] = pd.to_datetime(temp["time_tomerge"])
    ISO_dict[file.replace(".csv", "")] = temp

/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_14939/3786575145.py:3: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(os.path.join(FILE_PATH, file), thousands = ",")
/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_14939/3786575145.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(os.path.join(FILE_PATH, file), thousands = ",")
/var/folders/cz/nv3pr_214md_31815rhfvdkh0000gn/T/ipykernel_14939/3786575145.py:3: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv(os.path.join(FILE_PATH, file), thousands = ",")


In [11]:
[key for key in ISO_dict]

#add year, month, hour cols
for key in ISO_dict:
    ISO_dict[key]["year"] = ISO_dict[key]["time_tomerge"].apply(lambda x: x.year)
    ISO_dict[key]["month"] = ISO_dict[key]["time_tomerge"].apply(lambda x: x.month)
    ISO_dict[key]["hour"] = ISO_dict[key]["time_tomerge"].apply(lambda x: x.hour)

In [12]:
print(ISO_dict["PJM"].groupby(["year"])["load"].sum())

year
2014    7.973646e+08
2015    7.925906e+08
2016    7.923114e+08
2017    7.736165e+08
2018    8.065833e+08
2019    7.898233e+08
2020    7.572815e+08
2021    7.826954e+08
2022    7.950916e+08
2023    7.542295e+04
Name: load, dtype: float64


#clean CAISO
ISO_dict["CAISO"] = ISO_dict["CAISO"].fillna(0)
ISO_dict["CAISO"]["gen_solar_total"] = ISO_dict["CAISO"]["gen_solar"] + ISO_dict["CAISO"]["gen_solar_pv"] +ISO_dict["CAISO"]["gen_solar_thermal"]
ISO_dict["CAISO"]["gen_natural_gas"] = ISO_dict["CAISO"]["gen_thermal"] + ISO_dict["CAISO"]["gen_natural_gas"]

ISO_dict["CAISO"] = ISO_dict["CAISO"].drop(columns=["gen_solar", "gen_solar_pv", "gen_solar_thermal", "gen_renewables", "gen_thermal"])

In [4]:
positions = {"CAISO": [1,1], "ERCOT": [1,2], "PJM": [2,1], "ISONE": [2,2]}
print(plotly.colors.qualitative.Plotly)
colors = {}
for i,gen in enumerate(["gas", "imports", "solar", "wind", "nuclear", "hydro", "biomass", "biogas", "batteries", "coal"]):
    colors[gen] = plotly.colors.qualitative.Plotly[i]
print(colors)

['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
{'gas': '#636EFA', 'imports': '#EF553B', 'solar': '#00CC96', 'wind': '#AB63FA', 'nuclear': '#FFA15A', 'hydro': '#19D3F3', 'biomass': '#FF6692', 'biogas': '#B6E880', 'batteries': '#FF97FF', 'coal': '#FECB52'}


In [36]:
# import graph_objects from plotly package
import plotly.graph_objects as go

# import make_subplots function from plotly.subplots
# to make grid of plots
from plotly.subplots import make_subplots

def plot_gen(ISO_dict: dict, filepath: str):
    isos = ["CAISO", "ERCOT", "PJM", "ISONE"]
    solar_dict = {}
    fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(isos),specs=[[{"secondary_y": True}, {"secondary_y": True}], [{"secondary_y": True}, {"secondary_y": True}]])
    for iso in isos:

        try:
            ISO = ISO_dict[iso][[col for col in ISO_dict[iso].columns if "gen" in col]+["time_tomerge", "hour" , "month", "year"]]
        except KeyError:
            ISO = ISO_dict[iso][[col for col in ISO_dict[iso].columns if "gen_rt" in col]+["time_tomerge", "hour" , "month", "year"]]
            ISO.columns = [col.replace("rt_", "") for col in ISO.columns]
        ISO = ISO.drop_duplicates()
        ISO = ISO.reset_index()
        ISO = ISO[ISO["year"]!=2023]

        long = pd.wide_to_long(ISO, ['gen_'],i='index',j='gen_type',suffix='\w+').rename(columns={"gen_": "MWh"})
        grouped_long = long.groupby(["year", "gen_type"])["MWh"].sum().reset_index()
        average_gen = np.mean(grouped_long.groupby(["year"])["MWh"].sum())
        #years_to_keep = list(np.unique(grouped_long["year"])[grouped_long.groupby(["year"])["MWh"].sum()>average_gen*.5])
        #grouped_long = grouped_long[grouped_long["year"].isin(years_to_keep)]
        year_totals = pd.DataFrame(grouped_long.groupby(["year"])["MWh"].sum().reset_index())
        year_avg = year_totals.mean()
        print(year_totals)
        #grouped_long = grouped_long[grouped_long["MWh"]>year_avg*.01]
        grouped_long = grouped_long[grouped_long["gen_type"] == "solar"]
        print(type(year_totals))
        grouped_long["Percent"] = grouped_long["MWh"].div(year_totals["MWh"].values)
        #grouped_long["date"] = pd.to_datetime(grouped_long[['year', 'MONTH']].assign(DAY=1))
        print(grouped_long)
        grouped_long = grouped_long[grouped_long["year"]>2015]
        grouped_long["diff"] = grouped_long['MWh']-np.min(grouped_long["MWh"])
        solar_dict[iso] = grouped_long
        fig.add_trace(
        go.Scatter(x=grouped_long["year"], y=grouped_long["MWh"], name=f"MWh of yearly solar generation in {iso}"),
        secondary_y=False, row=positions[iso][0], col=positions[iso][1])
        fig.add_trace(
        go.Scatter(x=grouped_long["year"], y=grouped_long["Percent"], name=f"% of generation in {iso} from solar"),
        secondary_y=True, row=positions[iso][0], col=positions[iso][1])
        fig.update_xaxes(title_text="Year", row=positions[iso][0], col=positions[iso][1])
        fig.update_yaxes(title_text="Annual Solar Generation (MWh)", secondary_y=False, row=positions[iso][0], col=positions[iso][1])
        fig.update_yaxes(title_text="Percent of Annual Generation from Solar (%)", secondary_y=True, row=positions[iso][0], col=positions[iso][1])

    fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                      plot_bgcolor = "rgba(0,0,0,0)", font_color="black")
    fig.update_layout(height=1000, width=1400,
                  title_text="Total Annual Solar Generation from 2014 - 2022 by ISO")
    fig.update_yaxes(secondary_y=False, showgrid=False)
    fig.update_yaxes(secondary_y=True, showgrid=True)

    fig.show()

    os.makedirs(f"{filepath}", exist_ok=True)
    fig.write_html(os.path.join(filepath,f"full_solar_gen_old.html"))
    return solar_dict

    #fig.add_trace(go.area(grouped_long, x="year", y="MWh", color="gen_type",  color_discrete_sequence=plotly.colors.qualitative.Light24,
    #labels={
    #                 "MWh": "Total Generation (MWh)",
     #                "year": "Year",
     #                "gen_type": "Generation Type"
     #            },
      #          title=f"Total Annual Solar Generation for {iso} from 2014 - 2022")
    #fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
    #              plot_bgcolor = "rgba(0,0,0,0)")
    #fig.show()
    #os.makedirs(f"{filepath}", exist_ok=True)

    #fig.write_html(os.path.join(filepath,f"{iso}_full_annual.html"))

In [37]:
ling = plot_gen(ISO_dict, "/Users/evanlipton/Desktop/Thesis/generation_figures")

   year           MWh
0  2014  2.317407e+08
1  2015  2.324678e+08
2  2016  2.310887e+08
3  2017  2.311171e+08
4  2018  2.263220e+08
5  2019  2.187112e+08
6  2020  2.220499e+08
7  2021  2.244944e+08
8  2022  2.314358e+08
<class 'pandas.core.frame.DataFrame'>
     year gen_type           MWh   Percent
11   2014    solar  1.141813e+07  0.049271
24   2015    solar  1.563864e+07  0.067272
37   2016    solar  2.072511e+07  0.089685
50   2017    solar  2.534031e+07  0.109643
63   2018    solar  2.778036e+07  0.122747
76   2019    solar  2.860906e+07  0.130808
89   2020    solar  3.043528e+07  0.137065
102  2021    solar  3.437330e+07  0.153114
115  2022    solar  3.758854e+07  0.162415
   year           MWh
0  2014  8.471050e+07
1  2015  8.681665e+07
2  2016  8.809616e+07
3  2017  8.959435e+07
4  2018  9.400472e+07
5  2019  9.583640e+07
6  2020  9.513483e+07
7  2021  9.788658e+07
8  2022  1.071082e+08
<class 'pandas.core.frame.DataFrame'>
    year gen_type           MWh   Percent
7   2014    

In [42]:
fig = go.Figure()
fig = go.Figure()

iso_colors = {'CAISO': '#16FF32', 'ERCOT': '#FD3216', 'ISONE': '#3283FE', 'PJM': '#FEAF16'}

for key in ling:

    fig.add_trace(go.Scatter(x = ling[key]["year"], y = ling[key]["diff"], name = key, marker=dict(color = iso_colors[key])))
fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Annual Solar Generation (MWh)")


fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                  plot_bgcolor = "rgba(0,0,0,0)", font_color="black")
fig.update_layout(height=1000, width=1400,
              title_text="Increase in Annual Solar Generation from 2016 - 2022 by ISO")
fig.show()
fig.write_html(os.path.join("/Users/evanlipton/Downloads/new_solar.html"))



In [14]:
# import graph_objects from plotly package
import plotly.graph_objects as go

# import make_subplots function from plotly.subplots
# to make grid of plots
from plotly.subplots import make_subplots

def plot_makeup(ISO_dict: dict, filepath: str):
    isos = ["CAISO", "ERCOT", "PJM", "ISONE"]
    fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(isos), specs=[[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]])
    for iso in isos:

        try:
            ISO = ISO_dict[iso][[col for col in ISO_dict[iso].columns if "gen" in col]+["time_tomerge", "hour" , "month", "year"]]
        except KeyError:
            ISO = ISO_dict[iso][[col for col in ISO_dict[iso].columns if "gen_rt" in col]+["time_tomerge", "hour" , "month", "year"]]
            ISO.columns = [col.replace("rt_", "") for col in ISO.columns]
        ISO = ISO[[col for col in ISO.columns if ("landfill" not in col) & ("old" not in col) & ("refuse" not in col) & ("oil" not in col) & ("other" not in col) & ("multiple" not in col) & ("wood" not in col) & ("small" not in col)]]
        if "gen_natural_gas" in ISO.columns:
            print("natural_gas")
            ISO = ISO.rename(columns={"gen_natural_gas":"gen_gas"})
        if "gen_storage" in ISO.columns:
            ISO = ISO.rename(columns={"gen_storage":"gen_batteries"})
        if "gen_wsl" in ISO.columns:
            ISO = ISO.rename(columns={"gen_wsl":"gen_batteries"})
        if "gen_gascc" in ISO.columns:
            ISO["gen_gas"] = ISO["gen_gas"]+ ISO["gen_gascc"]
            ISO = ISO.drop(columns=["gen_gascc"])

        ISO = ISO.drop_duplicates()
        ISO = ISO.reset_index()
        ISO = ISO[ISO["year"]!=2023]

        long = pd.wide_to_long(ISO, ['gen_'],i='index',j='gen_type',suffix='\w+').rename(columns={"gen_": "MWh"})
        grouped_long = long.groupby(["year", "gen_type"])["MWh"].sum().reset_index()
        average_gen = np.mean(grouped_long.groupby(["year"])["MWh"].sum())
        years_to_keep = list(np.unique(grouped_long["year"])[grouped_long.groupby(["year"])["MWh"].sum()>average_gen*.5])
        grouped_long = grouped_long[grouped_long["year"].isin(years_to_keep)]
        year_totals = pd.DataFrame(grouped_long.groupby(["year"])["MWh"].sum().reset_index())
        year_avg = year_totals.mean()
        #grouped_long = grouped_long[grouped_long["MWh"]>year_avg*.01]
        grouped_long = grouped_long[grouped_long["year"] == 2022]
        grouped_long["Percent"] = grouped_long["MWh"]/np.sum(grouped_long["MWh"])


        fig.add_trace(
        go.Bar(base = grouped_long["Percent"], labels =grouped_long["gen_type"]), row=positions[iso][0], col=positions[iso][1])

    fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                      plot_bgcolor = "rgba(0,0,0,0)", font_color="black")
    fig.update_layout(height=1000, width=1400,
                  title_text="Make-up of Annual Generation for 2022 by ISO")
    fig.update_yaxes(secondary_y=False, showgrid=False)
    fig.update_yaxes(secondary_y=True, showgrid=True)

    fig.show()

    os.makedirs(f"{filepath}", exist_ok=True)
    fig.write_html(os.path.join(filepath,f"makeup_gen_new.html"))
    return grouped_long

    #fig.add_trace(go.area(grouped_long, x="year", y="MWh", color="gen_type",  color_discrete_sequence=plotly.colors.qualitative.Light24,
    #labels={
    #                 "MWh": "Total Generation (MWh)",
     #                "year": "Year",
     #                "gen_type": "Generation Type"
     #            },
      #          title=f"Total Annual Solar Generation for {iso} from 2014 - 2022")
    #fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
    #              plot_bgcolor = "rgba(0,0,0,0)")
    #fig.show()
    #os.makedirs(f"{filepath}", exist_ok=True)

    #fig.write_html(os.path.join(filepath,f"{iso}_full_annual.html"))

In [179]:
# import graph_objects from plotly package
import plotly.graph_objects as go

# import make_subplots function from plotly.subplots
# to make grid of plots
from plotly.subplots import make_subplots

def plot_hourly_gen(ISO_dict: dict, filepath: str):
    isos = ["CAISO", "ERCOT", "PJM", "ISONE"]
    fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(isos),)
    for iso in isos:

        try:
            ISO = ISO_dict[iso][[col for col in ISO_dict[iso].columns if "gen" in col]+["time_tomerge", "hour" , "month", "year"]]
        except KeyError:
            ISO = ISO_dict[iso][[col for col in ISO_dict[iso].columns if "gen_rt" in col]+["time_tomerge", "hour" , "month", "year"]]
            ISO.columns = [col.replace("rt_", "") for col in ISO.columns]

        ISO = ISO[[col for col in ISO.columns if ("landfill" not in col) & ("old" not in col) & ("refuse" not in col) & ("oil" not in col) & ("other" not in col) & ("multiple" not in col) & ("wood" not in col) & ("small" not in col)]]
        if "gen_natural_gas" in ISO.columns:
            print("natural_gas")
            ISO = ISO.rename(columns={"gen_natural_gas":"gen_gas"})
        if "gen_storage" in ISO.columns:
            ISO = ISO.rename(columns={"gen_storage":"gen_batteries"})
        if "gen_wsl" in ISO.columns:
            ISO = ISO.rename(columns={"gen_wsl":"gen_batteries"})
        if "gen_gascc" in ISO.columns:
            ISO["gen_gas"] = ISO["gen_gas"]+ ISO["gen_gascc"]
            ISO = ISO.drop(columns=["gen_gascc"])
        ISO = ISO.drop_duplicates()
        ISO = ISO.reset_index()
        ISO = ISO[ISO["year"]!=2023]

        long = pd.wide_to_long(ISO, ['gen_'],i='index',j='gen_type',suffix='\w+').rename(columns={"gen_": "MWh"})
        long = long[long["year"]==2022]
        grouped_long = long.groupby(["hour", "gen_type"])["MWh"].mean().reset_index()
        #average_gen = np.mean(grouped_long.groupby(["year"])["MWh"].sum())
        #years_to_keep = list(np.unique(grouped_long["year"])[grouped_long.groupby(["year"])["MWh"].sum()>average_gen*.5])
        #grouped_long = grouped_long[grouped_long["year"].isin(years_to_keep)]
        #year_totals = pd.DataFrame(grouped_long.groupby(["year"])["MWh"].sum().reset_index())
        #year_avg = year_totals.mean()
       # print(year_totals)
        #grouped_long = grouped_long[grouped_long["MWh"]>year_avg*.01]
        print(grouped_long)

        for i, gen_type in enumerate(np.unique(grouped_long["gen_type"])):
            if gen_type != "solar":
                fig.add_trace(
                go.Scatter(x=grouped_long["hour"][grouped_long["gen_type"] == gen_type], y=grouped_long["MWh"][grouped_long["gen_type"] == gen_type], stackgroup = "one", name = gen_type, line = dict(color=colors[gen_type])),
                row=positions[iso][0], col=positions[iso][1])
        gen_type = "solar"

        fig.add_trace(
        go.Scatter(x=grouped_long["hour"][grouped_long["gen_type"] == gen_type], y=grouped_long["MWh"][grouped_long["gen_type"] == gen_type], stackgroup = "one", name = gen_type, line = dict(color=colors[gen_type])),
        row=positions[iso][0], col=positions[iso][1])





        fig.update_xaxes(title_text="Hour", row=positions[iso][0], col=positions[iso][1])
        fig.update_yaxes(title_text="Generation (MWh)", row=positions[iso][0], col=positions[iso][1])

    fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
                      plot_bgcolor = "rgba(0,0,0,0)", font_color="black")
    fig.update_layout(height=1000, width=1400,
                  title_text="Average Hourly Generation Profile from 2022 by ISO")

    fig.show()

    os.makedirs(f"{filepath}", exist_ok=True)
    fig.write_html(os.path.join(filepath,f"hourly_gen_profile_new.html"))
    return grouped_long

    #fig.add_trace(go.area(grouped_long, x="year", y="MWh", color="gen_type",  color_discrete_sequence=plotly.colors.qualitative.Light24,
    #labels={
    #                 "MWh": "Total Generation (MWh)",
     #                "year": "Year",
     #                "gen_type": "Generation Type"
     #            },
      #          title=f"Total Annual Solar Generation for {iso} from 2014 - 2022")
    #fig.update_layout(paper_bgcolor = "rgba(0,0,0,0)",
    #              plot_bgcolor = "rgba(0,0,0,0)")
    #fig.show()
    #os.makedirs(f"{filepath}", exist_ok=True)

    #fig.write_html(os.path.join(filepath,f"{iso}_full_annual.html"))

In [180]:
ling2 = plot_hourly_gen(ISO_dict, "/Users/evanlipton/Desktop/Thesis/generation_figures")


natural_gas
     hour   gen_type          MWh
0       0  batteries    63.528311
1       0     biogas   201.887900
2       0    biomass   286.917123
3       0       coal     8.608447
4       0        gas  9951.968493
..    ...        ...          ...
235    23      hydro  1741.305495
236    23    imports  7232.293361
237    23    nuclear  2016.324084
238    23      solar    -9.040018
239    23       wind  2823.091438

[240 rows x 3 columns]
     hour   gen_type          MWh
0       0  batteries   -18.454247
1       0    biomass    15.916233
2       0       coal  1916.620685
3       0        gas  4319.820342
4       0      hydro     6.254863
..    ...        ...          ...
187    23        gas  4823.540479
188    23      hydro     3.241849
189    23    nuclear  1188.888288
190    23      solar     0.011507
191    23       wind  3681.777534

[192 rows x 3 columns]
     hour   gen_type           MWh
0       0  batteries      0.000000
1       0       coal  19149.902210
2       0        ga

In [75]:
for ISO in ["CAISO", "ISONE", "PJM", "ERCOT"]:
    plot_gen(ISO_dict, ISO, "/Users/evanlipton/Desktop/Thesis/generation_figures")

In [38]:
print(results.sort_values(by="MWh").head(70))

     year   gen_type           MWh
91   2021  batteries -8.454450e+04
65   2019  batteries -2.707436e+04
78   2020  batteries -1.981125e+04
0    2014  batteries  0.000000e+00
42   2017       coal  0.000000e+00
..    ...        ...           ...
5    2014      hydro  1.195543e+07
25   2015       wind  1.223793e+07
18   2015      hydro  1.236245e+07
12   2014       wind  1.288483e+07
109  2022      hydro  1.312597e+07

[70 rows x 3 columns]


In [162]:
ISO = ISO_dict["CAISO"]

In [165]:
ISO.groupby(["hour", "year"])[""].mean()

hour  year
0     2014    22961.956164
      2015    23209.701370
      2016    23160.368852
      2017    23352.558904
      2018    23522.876712
                  ...     
23    2018    25044.736986
      2019    25443.731507
      2020    25350.363388
      2021    25699.424658
      2022    26324.071233
Name: load_rt, Length: 217, dtype: float64

In [166]:
ISO_dict["CAISO"]

,time_tomerge,ancillary_da_nr,ancillary_da_rd,ancillary_da_rmd,ancillary_da_rmu,ancillary_da_ru,ancillary_da_sr,ancillary_rt_nr,ancillary_rt_rd,ancillary_rt_rmd,...,congestion_rt,energy_rt,losses_rt,gen_coal,gen_natural_gas,gen_batteries,gen_other,year,month,gen_solar_total
0,2014-01-01 00:00:00+00:00,40.0984,625.40,3.9089,1180.9560,37.89900,59.89400,3.747040,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,12461.000000,0.000000,0.0,2014,1,0.000000
1,2014-01-01 01:00:00+00:00,47.2970,681.45,0.0000,698.1880,33.18600,61.78100,4.713687,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,12163.000000,0.000000,0.0,2014,1,0.000000
2,2014-01-01 02:00:00+00:00,44.8790,769.95,0.0000,0.0000,552.90000,64.04900,4.749625,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,11347.000000,0.000000,0.0,2014,1,0.000000
3,2014-01-01 03:00:00+00:00,43.5180,769.95,0.0000,6.7191,550.05000,62.68800,4.941813,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,11089.000000,0.000000,0.0,2014,1,0.000000
4,2014-01-01 04:00:00+00:00,42.9170,769.95,0.0000,18.8091,743.05000,62.08700,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,11118.000000,0.000000,0.0,2014,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78884,2022-12-31 20:00:00+00:00,101.5209,5700.00,0.0000,0.0000,2483.99900,2533.73540,0.000000,106.494525,0.0,...,0.0,93.035366,-3.333124,3.250000,6293.083333,-404.416667,0.0,2022,12,4607.916667
78885,2022-12-31 21:00:00+00:00,90.5520,6612.00,0.0000,0.0000,1892.67000,1916.61000,0.000000,214.631892,0.0,...,0.0,106.991223,-3.714072,3.500000,7216.833333,-169.750000,0.0,2022,12,3530.500000
78886,2022-12-31 22:00:00+00:00,87.7104,8850.00,0.0000,0.0000,4625.81000,3186.47000,0.000000,0.000000,0.0,...,0.0,106.177040,-3.627994,3.833333,7848.750000,2.500000,0.0,2022,12,2455.750000
78887,2022-12-31 23:00:00+00:00,84.4800,5670.00,0.0000,0.0000,1040.81446,845.73324,0.000000,162.101250,0.0,...,0.0,115.759343,-3.939707,4.083333,8740.833333,256.750000,0.0,2022,12,1174.250000
